In [ ]:
%%time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%%time
!pip uninstall pylibcugraph-cu12 -y

In [ ]:
%%time
# !pip install ebooklib beautifulsoup4 kokoro>=0.9.2 soundfile torch pydub
!pip install ebooklib beautifulsoup4 kokoro>=0.9.2 soundfile torch pydub pylibraft-cu12==24.12.0 rmm-cu12==24.12.0

In [ ]:
%%time
!pip uninstall pylibcugraph-cu12 -y

In [ ]:
%%time
# !pip install ebooklib beautifulsoup4 kokoro>=0.9.2 soundfile torch pydub
!pip install ebooklib beautifulsoup4 kokoro>=0.9.2 soundfile torch pydub pylibraft-cu12==24.12.0 rmm-cu12==24.12.0

In [ ]:
%%time
!sudo apt-get install ffmpeg

In [ ]:
%%time
import os
import logging
import subprocess
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
from kokoro import KPipeline
import soundfile as sf
import torch
import numpy as np
from pydub import AudioSegment

In [ ]:
%%time
class AudiobookCreator:
    def __init__(self, epub_path, output_dir="audiobooks", voice='af_heart'):
        """Initialize the AudiobookCreator with EPUB path and output settings."""
        self.epub_path = epub_path
        self.epub_name = os.path.splitext(os.path.basename(epub_path))[0]
        self.output_dir = os.path.join(output_dir, self.epub_name)
        self.txt_dir = os.path.join(self.output_dir, "txt")
        self.audio_dir = os.path.join(self.output_dir, "audio")
        self.voice = voice
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.pipeline = KPipeline(lang_code='a', device=self.device)
        self.chapter_names = []
        
        # Set up logging
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setFormatter(logging.Formatter('%(levelname)s: %(message)s'))
        self.logger.addHandler(handler)

    def extract_chapters(self):
        """Extract chapters from EPUB and save as text files with chapter names."""
        os.makedirs(self.txt_dir, exist_ok=True)
        try:
            book = epub.read_epub(self.epub_path)
        except Exception as e:
            self.logger.error(f"Failed to read EPUB file: {e}")
            return

        # Extract chapter names from TOC
        for item in book.toc:
            if isinstance(item, epub.Link):
                self.chapter_names.append(item.title)

        chapter_count = 0
        for item in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
            chapter_count += 1
            content = item.get_content().decode('utf-8')
            soup = BeautifulSoup(content, 'html.parser')
            text = soup.get_text(separator='\n', strip=True)
            chapter_name = (self.chapter_names[chapter_count - 1] 
                          if chapter_count <= len(self.chapter_names) 
                          else f"Chapter {chapter_count}")
            chapter_file = os.path.join(self.txt_dir, f"{chapter_name}.txt")
            try:
                with open(chapter_file, 'w', encoding='utf-8') as f:
                    f.write(text)
                self.logger.info(f"Saved chapter text: {chapter_file}")
            except Exception as e:
                self.logger.error(f"Error writing {chapter_file}: {e}")

    def text_to_audio(self):
        """Convert text files to audio using Kokoro TTS."""
        os.makedirs(self.audio_dir, exist_ok=True)
        txt_files = [f for f in os.listdir(self.txt_dir) if f.endswith('.txt')]
        for txt_file in txt_files:
            chapter_name = os.path.splitext(txt_file)[0]
            chapter_path = os.path.join(self.txt_dir, txt_file)
            audio_path = os.path.join(self.audio_dir, f"{chapter_name}.wav")
            try:
                with open(chapter_path, "r", encoding="utf-8") as f:
                    text = f.read()
                generator = self.pipeline(text, voice=self.voice)
                combined_audio = []
                for _, _, audio in generator:
                    combined_audio.append(audio)
                combined_audio = np.concatenate(combined_audio)
                sf.write(audio_path, combined_audio, 24000)
                self.logger.info(f"Saved audio: {audio_path}")
            except Exception as e:
                self.logger.error(f"Error converting {chapter_path} to audio: {e}")

    def create_audiobook(self):
        """Combine audio files into a single M4B audiobook with chapters."""
        audio_files = sorted([f for f in os.listdir(self.audio_dir) if f.endswith(".wav")])
        combined = AudioSegment.empty()
        chapters = []
        current_time = 0

        # Combine audio and track chapter timings
        for audio_file in audio_files:
            chapter_name = os.path.splitext(audio_file)[0]
            audio_path = os.path.join(self.audio_dir, audio_file)
            try:
                segment = AudioSegment.from_wav(audio_path)
                combined += segment
                chapters.append((current_time, chapter_name))
                current_time += len(segment) / 1000  # Convert ms to seconds
            except Exception as e:
                self.logger.error(f"Error processing {audio_path}: {e}")

        # Export initial combined audio
        combined_path = os.path.join(self.output_dir, f"{self.epub_name}.m4b")
        try:
            combined.export(combined_path, format="ipod", tags={"album": self.epub_name, "artist": "Author"})
        except Exception as e:
            self.logger.error(f"Error exporting combined audio: {e}")
            return

        # Write chapter metadata file
        chapters_file = os.path.join(self.output_dir, "chapters.txt")
        try:
            with open(chapters_file, "w") as f:
                for i, (start_time, name) in enumerate(chapters):
                    f.write(f"CHAPTER{i+1}={start_time:.3f}\n")
                    f.write(f"CHAPTER{i+1}NAME={name}\n")
        except Exception as e:
            self.logger.error(f"Error writing chapters file: {e}")
            return

        # Use ffmpeg to add chapters
        final_path = os.path.join(self.output_dir, f"{self.epub_name}_final.m4b")
        cmd = [
            "ffmpeg", "-i", combined_path, "-i", chapters_file,
            "-map_metadata", "1", "-codec", "copy", final_path, "-y"
        ]
        try:
            subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            self.logger.info(f"Audiobook created: {final_path}")
        except subprocess.CalledProcessError as e:
            self.logger.error(f"FFmpeg error: {e.stderr.decode()}")

    def run(self):
        """Execute the full audiobook creation process."""
        self.extract_chapters()
        self.text_to_audio()
        self.create_audiobook()

In [ ]:
%%time
# Example usage
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)
    creator = AudiobookCreator("/kaggle/input/ebooks-epubs/sample_01.epub")
    creator.run()

In [ ]:
# %%time
# # Example usage
# if __name__ == "__main__":
#     logging.basicConfig(level=logging.INFO)
#     creator = AudiobookCreator("/kaggle/input/ebooks-epubs/sample_01.epub")
#     creator.run()